In [1]:
import os
from langchain_community.llms import Ollama
from langchain.prompts import ChatPromptTemplate, PromptTemplate, HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage, HumanMessage
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

In [2]:
ollama_url = os.getenv('OLLAMA_URL')
model = Ollama(base_url=ollama_url, model="dolphin-mistral", keep_alive=5, temperature=0.0)

ValidationError: 1 validation error for Ollama
base_url
  none is not an allowed value (type=type_error.none.not_allowed)

In [ ]:
title_schema = [ResponseSchema(name="title", description="Title for new story")]
   
output_parser = StructuredOutputParser.from_response_schemas(title_schema)

In [ ]:
prompt = PromptTemplate(
	template="Title: {title}\nSummary: {summary}\n{format_instructions}\n",
	input_variables=["title", "summary"],
	partial_variables={"format_instructions": output_parser.get_format_instructions()},
)

# format chat prompt
system_prompt = SystemMessage(content=("""
		You are and news article title editor that reviews and provides a concise and accurate title when given an existing Title and article Summary. 
		Remove all links from the title.
		Title should be as short as possible, aim to be less that 70 characters long.
		Title should have an absolute minimum of punctuation.
		Do your best to keep the existing title if possible.
		"""))
user_prompt = HumanMessagePromptTemplate(prompt=prompt)

chat_prompt = ChatPromptTemplate.from_messages([system_prompt, user_prompt])

In [ ]:
chain = chat_prompt | model

In [ ]:
items = [
{'title': 'OPEN THREAD:  Just do it.', 'summary': 'OPEN THREAD: Just do it.'},
{'title': 'THIS PARAGRAPH CAPTURES THE ESSENCE OF THE ATLANTIC’S SAD PIECE ON THE WORLD’S BIGGEST CRUISE SHIP:', 'summary': 'THIS PARAGRAPH CAPTURES THE ESSENCE OF THE ATLANTIC&#8217;S SAD PIECE ON THE WORLD&#8217;S BIGGEST CRUISE SHIP: &#8220;A real journalist or a more intrepid conversationalist would have gone up to the couple and asked them to explain the longevity of their marriage vis-à-vis their love of cruising. But instead I head to my mall suite, take [&#8230;]'},
{'title': 'THE NEW SPACE RACE:  China launches first of a new series of Yaogan reconnaissance satellites.', 'summary': 'THE NEW SPACE RACE: China launches first of a new series of Yaogan reconnaissance satellites.'},
{'title': 'WHY IS THE BIDEN ADMINISTRATION TRYING SO HARD TO KEEP HAMAS FROM LOSING?  Biden Calls for Ceasefire', 'summary': 'WHY IS THE BIDEN ADMINISTRATION TRYING SO HARD TO KEEP HAMAS FROM LOSING? Biden Calls for Ceasefire During Phone Call with Netanyahu.'},
{'title': 'BOOK RECOMMENDATION:  So I just finished up The Naval Odyssey of Prof. James Brand, so far as it goe', 'summary': 'BOOK RECOMMENDATION: So I just finished up The Naval Odyssey of Prof. James Brand, so far as it goes to date, and quite enjoyed it. It&#8217;s a World War II series with plenty of action, but its central character is Prof. James Brand, a wunderkind physicist/operations analyst who zips around solving problems with everything from [&#8230;]'},
{'title': 'TUNED MASS DAMPERS FOR THE WIN:  How a steel ball protected Taiwan’s tallest skyscraper in an eart', 'summary': 'TUNED MASS DAMPERS FOR THE WIN: How a steel ball protected Taiwan’s tallest skyscraper in an earthquake.'},
{'title': 'SCIENCE:  Thinking You Look Younger Has A Curious Link to How You Age.', 'summary': 'SCIENCE: Thinking You Look Younger Has A Curious Link to How You Age.'},
{'title': 'ATTEMPT NO LANDINGS THERE:  Jupiter’s Moon Europa Beckons As NASA’s Clipper Prepares for the Unk', 'summary': 'ATTEMPT NO LANDINGS THERE: Jupiter’s Moon Europa Beckons As NASA’s Clipper Prepares for the Unknown.'},
{'title': 'FLASHBACK:  Time for Consequences:  The “Cabal” who bragged about rigging the 2020 election stuck us', 'summary': 'FLASHBACK: Time for Consequences: The &#8220;Cabal&#8221; who bragged about rigging the 2020 election stuck us with an incapable president at a time of crisis. Examples need to be made.'},
{'title': 'MICROBIOME NEWS:  Gut microbes may influence odds for obesity.', 'summary': 'MICROBIOME NEWS: Gut microbes may influence odds for obesity.'},
]

In [ ]:
for item in items:
  output = output_parser.parse(chain.invoke({"title": item['title'], "summary": item['summary']}))
  print(output['title'])



Just Do It: Open Thread
World's Biggest Cruise Ship: Couple's Long Marriage and Love of Cruising
China Launches First Yaogan Recon Satellite in New Series
Biden Calls for Ceasefire, Biden Admin Supports Hamas
Naval Odyssey: Enjoyed WWII Series with Prof. James Brand
Taiwan's Tallest Skyscraper Protected by Tuned Mass Dampers in Earthquake
Thinking Younger Linked to Aging Process
NASA's Clipper to Explore Europa
Cabal Rigged 2020 Election, Led to Incapable President in Crisis
Gut Microbes Influence Odds For Obesity
